1. 스타 데이터에서 유튜브 링크가 잘못된 개체들이 있음
2. 잘못된 유튜브 링크들을 올바른 링크로 업데이트하는 sql 쿼리문이 필요함 단,
 - 그룹 멤버인데 개인 채널이 없고 그룹 채널만 있는 경우? 그냥 넘어감
 - 공식 채널이 없는 경우? 넘어감
3. 잘못된 유튜브 링크 리스트들은 api(prod_오매핑 개체 리스트)에 접속해서 볼 수 있고, 검수 및 맵핑 후엔 해당 api에서 리스트들을 제거해야 함
4. 제거는 api(prod_오매핑 개체 checked 업데이트_body)에서 body에 검수한 series_id를 입력, send하면 제거됨
5. 제거 후, 작성한 sql 쿼리문을 종우님께 전달하면 끝!

### 잘못된 유튜브 주소 리스트 보기

[GET] ****

x-api-key: ****
### 잘못된 유튜브 주소 삭제 (서비스에 노출되는 테이블 아님)

[DELETE] ****

x-api-key: ****

- 삭제방법1: path param에 series_id 하나 추가 (...prod/url_checker/578)

- 삭제방법2: body에 series_id 리스트 추가
{
    "series_id": [787,100,1]
}

#### 잘못된 유튜브 채널 양식

https://www.youtube.com/watch?v=ladClnnJhqg&list=PL9K3xwFkFqWGvtbvCXqOJwCIGYgZMD_7-&index=12

#### 올바른 유튜브 채널 양식

https://www.youtube.com/channel/UC8HEl74jL3bLLwfDP1OALxw

- 대량 삭제가 필요할 시 저한테 요청해주세요

- 삭제할 양이 많지 않으면 donut관리자에서 직접 수정


### 검수한 리스트 제거
prod_오매핑 개체 checked 업데이트_body 에서 body에 검수한 series_id 입력

In [1]:
import json
import requests as req
import pandas as pd
import time
import datetime

save_dir = '/Users/jieun/Desktop/데이터 팀 인턴 이지은/유튜브 매핑/sql/'

t = datetime.datetime.now()
now = t.strftime('%Y%m%d')

In [2]:
# api 접속해서 유튜브 url 오매핑된 데이터 로드
url = ''
res = req.get(url,headers={'x-api-key':''})

j = json.loads(res.text)
j

[{'uye_pk': '1179',
  'cd_name': '희주',
  'series_id': '19851207',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/c/YoosBeauty/about',
  'input_channel_id': 'https://www.youtube.com/c/YoosBeauty/about',
  'regist_date': '2021-10-16 10:00:45',
  'update_date': '2021-10-16 10:00:45',
  'donut_url': 'http://dev.mycelebs.com/donut/Celeb/ShowManageCeleb/19851207',
  'checked': 'False'},
 {'uye_pk': '1180',
  'cd_name': '연두콩',
  'series_id': '19851208',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/yeondukong',
  'input_channel_id': 'https://www.youtube.com/yeondukong',
  'regist_date': '2021-10-16 10:00:46',
  'update_date': '2021-10-16 10:00:46',
  'donut_url': 'http://dev.mycelebs.com/donut/Celeb/ShowManageCeleb/19851208',
  'checked': 'False'},
 {'uye_pk': '1181',
  'cd_name': '다영',
  'series_id': '19851217',
  'status': '400',
  'db_csu_url': 'https://www.youtube.com/adrenallingb',
  'input_channel_id': 'https://www.youtube.com/adrenallingb',
  'regist_date': '

In [3]:
# 총 검수 갯수
len(j)

260

In [4]:
# 필요한 데이터만 가지고 와서,
bin_df = pd.DataFrame(j)

bin_df = bin_df.drop(['uye_pk', 'status', 'input_channel_id', 'regist_date', 'update_date', 'donut_url', 'checked'], axis=1)
bin_df

,cd_name,series_id,db_csu_url
0,희주,19851207,https://www.youtube.com/c/YoosBeauty/about
1,연두콩,19851208,https://www.youtube.com/yeondukong
2,다영,19851217,https://www.youtube.com/adrenallingb
3,임태빈,19866394,https://www.youtube.com/c/HELLOJACKIE
4,시오,19866568,https://www.youtube.com/c/시오ASMRSio/about
...,...,...,...
255,비비드,20869797,http://www.youtube.com/vividkpop
256,나타샤 레게로,20875877,https://www.youtube.com/msnatashaleggero
257,문남권,20891477,https://www.youtube.com/mooondaegi
258,신혁,20925009,https://www.youtube.com/user/shinhyuk0209


In [5]:
# input에 올바른 url 입력
# 없으면 그냥 엔터

list_ = []
url = []

for idx, row in bin_df.iterrows():
    print(row['cd_name'], row['series_id'], row['db_csu_url'])
    url_input = input('url를 입력하세요')
    url.append(url_input)
    
    youtube = {'cd_name':row['cd_name'], 'series_id':row['series_id'], 'db_csu_url':row['db_csu_url'], 'new_url':url_input}
    list_.append(youtube)

희주 19851207 https://www.youtube.com/c/YoosBeauty/about
url를 입력하세요https://www.youtube.com/c/YoosBeauty/featured
연두콩 19851208 https://www.youtube.com/yeondukong
url를 입력하세요https://www.youtube.com/c/yeondukong
다영 19851217 https://www.youtube.com/adrenallingb
url를 입력하세요https://www.youtube.com/c/lifeofkim
임태빈 19866394 https://www.youtube.com/c/HELLOJACKIE
url를 입력하세요
시오 19866568 https://www.youtube.com/c/시오ASMRSio/about
url를 입력하세요https://www.youtube.com/c/%EC%8B%9C%EC%98%A4ASMRSio/featured
교광 19866571 https://www.youtube.com/c/교광TVkgxhwang/about
url를 입력하세요https://www.youtube.com/c/%EA%B5%90%EA%B4%91TVkgxhwang/featured
랜달 파크 19908589 https://www.youtube.com/user/randall1
url를 입력하세요
참PD 19908639 https://www.youtube.com/user/youl2007
url를 입력하세요
제인 배들러 19923757 https://www.youtube.com/user/officialjanebadler
url를 입력하세요
유준성 19929880 https://www.youtube.com/c/DOZTV
url를 입력하세요
천뚱 19932002 https://www.youtube.com/watch?v=UWyah2YeoTU
url를 입력하세요https://www.youtube.com/channel/UCF8W68PS2Ltz17lTDSptDvg
요

url를 입력하세요NULL
황린 19127392 https://www.youtube.com/c/leeneyhwang
url를 입력하세요
한 메구미 19127417 https://www.youtube.com/megumihanfans2
url를 입력하세요
이서 19127433 https://www.youtube.com/JJOA_JPLANETENT
url를 입력하세요
태령 19127434 https://www.youtube.com/JJOA_JPLANETENT
url를 입력하세요
유우리 19127435 https://www.youtube.com/JJOA_JPLANETENT
url를 입력하세요
진화 19127439 https://www.youtube.com/BLITZERSOFFICIAL
url를 입력하세요
크리스 19127442 https://www.youtube.com/BLITZERSOFFICIAL
url를 입력하세요
샤 19127443 https://www.youtube.com/BLITZERSOFFICIAL
url를 입력하세요
루탄 19127444 https://www.youtube.com/BLITZERSOFFICIAL
url를 입력하세요
우주 19127445 https://www.youtube.com/BLITZERSOFFICIAL
url를 입력하세요
김세별 19142584 https://www.youtube.com/c/jiyoungs2s2
url를 입력하세요
이근행 19142653 https://www.youtube.com/user/newstapa/videos
url를 입력하세요https://www.youtube.com/c/newstapa
윤은오 19142661 https://www.youtube.com/c/VROMANCEofficial
url를 입력하세요
희찬 19142714 https://www.youtube.com/DKBOFFICIAL
url를 입력하세요
준서 19142715 https://www.youtube.com/DKBOFFICIAL
url를 입력하세요

url를 입력하세요
애쉬비 2508 https://www.youtube.com/waze00able
url를 입력하세요
나자닌 보니아디 20869715 https://www.youtube.com/user/NazaninBoniadi
url를 입력하세요
비비드 20869797 http://www.youtube.com/vividkpop
url를 입력하세요
나타샤 레게로 20875877 https://www.youtube.com/msnatashaleggero
url를 입력하세요
문남권 20891477 https://www.youtube.com/mooondaegi
url를 입력하세요
신혁 20925009 https://www.youtube.com/user/shinhyuk0209
url를 입력하세요
심짱 20925012 https://www.youtube.com/simzzang
url를 입력하세요


In [6]:
all_idx = []

for l in list_ :
    idx = l['series_id']
    all_idx.append(idx)
    
all_idx

['19851207',
 '19851208',
 '19851217',
 '19866394',
 '19866568',
 '19866571',
 '19908589',
 '19908639',
 '19923757',
 '19929880',
 '19932002',
 '19932006',
 '19932008',
 '19932063',
 '19932095',
 '20012550',
 '20012578',
 '20027682',
 '20027735',
 '20027740',
 '20027757',
 '20027760',
 '20027786',
 '20027787',
 '2834',
 '2836',
 '20058868',
 '20058876',
 '20058881',
 '281169',
 '281171',
 '288262',
 '20093116',
 '20093171',
 '2771',
 '20231645',
 '3256',
 '281118',
 '20325494',
 '20329624',
 '478627',
 '20353148',
 '573451',
 '12242023',
 '20395743',
 '20395886',
 '5213941',
 '664326',
 '19142717',
 '664308',
 '2856',
 '2857',
 '14565475',
 '20413111',
 '20428983',
 '2956',
 '2959',
 '284062',
 '2960',
 '20444478',
 '292926',
 '406',
 '18883733',
 '18883755',
 '2807',
 '1959',
 '2907',
 '2915',
 '18902645',
 '18905483',
 '18905509',
 '18955844',
 '18955871',
 '18955874',
 '18955886',
 '18955893',
 '18955912',
 '18955924',
 '2311890',
 '18958751',
 '18958806',
 '18958807',
 '18958810',


In [8]:
# 맵핑한 url들을 업데이트 쿼리문으로 작성해주는 코드
# 저장된 sql 파일 종우님께 전달

df = pd.DataFrame(list_, columns = ['cd_name', 'series_id', 'db_csu_url', 'new_url'])
list_idx = []

for idx, row in df.iterrows():
    new_url = row['new_url']
    series_id = row['series_id']
    
    if row['new_url'] == '' :
        pass
    else :
        query = f'''UPDATE `celeb_site_url` SET `csu_url` = '{new_url}', `update_date` = curdate() where `cd_idx` = {series_id};\n'''
        print(query)
        list_idx.append(series_id)
        
        with open(save_dir + now + '_youtube.sql', 'at') as f:
            f.write(query)
    

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/c/YoosBeauty/featured', `update_date` = curdate() where `cd_idx` = 19851207;

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/c/yeondukong', `update_date` = curdate() where `cd_idx` = 19851208;

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/c/lifeofkim', `update_date` = curdate() where `cd_idx` = 19851217;

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/c/%EC%8B%9C%EC%98%A4ASMRSio/featured', `update_date` = curdate() where `cd_idx` = 19866568;

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/c/%EA%B5%90%EA%B4%91TVkgxhwang/featured', `update_date` = curdate() where `cd_idx` = 19866571;

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/channel/UCF8W68PS2Ltz17lTDSptDvg', `update_date` = curdate() where `cd_idx` = 19932002;

UPDATE `celeb_site_url` SET `csu_url` = 'https://www.youtube.com/user/biryoungstyle/featured', `update_date` = curdat

In [9]:
# postman -> prod_오매핑 개체 checked 업데이트_body -> 검수한 리스트 제거하고
# sql 파일 첨부해서 보고

month = time.strftime('%m', time.localtime(time.time()))
day = time.strftime('%d', time.localtime(time.time()))

print('*{0}월 {1}일 스타 유튜브 검수 및 매핑 완료했습니다. @jjongwoo'.format(month, day))
print('- 검수 리스트 : {}명'.format(len(j)))
print(", ".join(all_idx), end='\n\n')
print('- 매핑 리스트 : {}명'.format(len(list_idx)))
print(", ".join(list_idx), end='\n\n')

*11월 30일 스타 유튜브 검수 및 매핑 완료했습니다. @jjongwoo
- 검수 리스트 : 260명
19851207, 19851208, 19851217, 19866394, 19866568, 19866571, 19908589, 19908639, 19923757, 19929880, 19932002, 19932006, 19932008, 19932063, 19932095, 20012550, 20012578, 20027682, 20027735, 20027740, 20027757, 20027760, 20027786, 20027787, 2834, 2836, 20058868, 20058876, 20058881, 281169, 281171, 288262, 20093116, 20093171, 2771, 20231645, 3256, 281118, 20325494, 20329624, 478627, 20353148, 573451, 12242023, 20395743, 20395886, 5213941, 664326, 19142717, 664308, 2856, 2857, 14565475, 20413111, 20428983, 2956, 2959, 284062, 2960, 20444478, 292926, 406, 18883733, 18883755, 2807, 1959, 2907, 2915, 18902645, 18905483, 18905509, 18955844, 18955871, 18955874, 18955886, 18955893, 18955912, 18955924, 2311890, 18958751, 18958806, 18958807, 18958810, 18958811, 18958812, 18958813, 18958814, 18958815, 18958816, 18958817, 18958818, 18958819, 18958820, 18958821, 11628242, 11628243, 11628255, 11628258, 11628263, 11628265, 18974074, 18974120, 1